In [334]:
import pandas as pd
import numpy as np
import pickle, json
import random
import time
import os


import tensorflow as tf
from tensorflow import keras

from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.datasets
from sklearn import preprocessing



from tqdm.notebook import tqdm

In [ ]:
import pandas as pd




In [1]:
def extraction_dataset(data_2,number):
    test_model = data_2[data_2["Model"]==number]
    a = list(test_model["LayerNum"].value_counts().sort_index().keys())
    a_index = [i-1 for i in a]
    df_ = test_model.iloc[a_index,:]
    return df_

In [16]:
data = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_PCB\data\Data_preprocessing5.xlsx")
data = data.fillna(0)
data.head()

,Model,LayerNum,PP,MatlMaker,RD,GD,MatlNum,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,1.36,2.56,1080.0,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,1.36,2.56,7628.0,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,1.36,2.56,3313.0,53.0,81.4,14.29,75,15.04,48,90,80.48


In [1021]:
data["MatlMaker"].value_counts()

DS7402M     1809
DS7402LC     635
EM-370Z      218
IT150G       126
EM-526       117
NPG-151      110
Name: MatlMaker, dtype: int64

In [17]:
dic_glass = {}
keys = data["MatlG"].groupby(data["MatlNum"]).mean().keys()
values = data["MatlG"].groupby(data["MatlNum"]).mean().values
for i in range(len(keys)):
    dic_glass[keys[i]] = round(values[i],2)
data = data.drop(columns=["RD","GD","MatlNum"])
data.head()


,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,53.0,81.4,14.29,75,15.04,48,90,80.48


In [18]:
model_list = list(data["Model"].value_counts().keys())
testset_dic = {}
testset_list = []
for mo in model_list:
    globals () ["model_{}".format(mo)] = extraction_dataset(data,mo)
    testset_list.append(globals () ["model_{}".format(mo)])
    testset_dic["model_{}".format(mo)] = globals () ["model_{}".format(mo)]
drop_index = []
for i in testset_list:
    drop_index.extend(list(i.index))
data_drop = data.drop(drop_index)

In [19]:
data_drop.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
11,1,1,1,DS7402M,61.0,46.8,8.37,55,14.29,73,60,66.19
12,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,74.47
13,1,3,1,DS7402M,47.0,203.4,15.04,56,15.04,46,200,196.32
14,1,4,0,DS7402M,0.0,0.0,15.04,46,15.04,75,75,75.22
15,1,5,1,DS7402M,53.0,81.4,15.04,75,15.04,48,90,82.26


In [20]:
testset_list[0].head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
235,5,1,1,EM-370Z,63.0,46.8,8.14,50,14.91,70,60,68.590
236,5,2,0,EM-370Z,58.0,48.0,14.91,70,14.91,51,64,60.390
237,5,3,1,EM-370Z,57.0,103.8,14.91,51,0.00,0,100,119.285
238,5,4,1,EM-370Z,57.0,103.8,0.00,0,15.66,40,100,119.285
239,5,5,0,EM-370Z,58.0,48.0,15.66,40,14.17,72,64,59.640


In [21]:
data_drop = data_drop[data_drop["PP"]==1]
Maker = data_drop['MatlMaker'].unique()
Glass_all = data_drop['MatlG'].unique()
len_list = {}

for i in Maker:
    df = data_drop[data_drop['MatlMaker'] == i]
    Glass = df['MatlG'].unique()
    for j in Glass:
        df2 = df[df['MatlG'] == j]
        len_list["{} {}".format(i,j)] = len(df2)
list(len_list.keys())[list(len_list.values()).index(max(len_list.values()))]

'DS7402M 46.8'

# 'DS7402M 46.8' 로 test 하기

In [347]:
data_test = data_drop[data_drop["MatlMaker"]=="DS7402M"]
data_test = data_drop[data_drop["MatlG"]==46.8]

drop_list = ['MatlMaker','MatlG',"Model","PP"]
data_test = data_test.drop(columns=drop_list)
data_test.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
11,1,61.0,8.37,55,14.29,73,60,66.19
21,11,61.0,14.29,73,8.53,56,60,65.44
22,1,61.0,8.69,55,14.29,73,60,67.70
32,11,61.0,14.29,73,8.05,56,60,66.02
33,1,61.0,8.37,55,13.54,73,60,67.70


In [941]:
data_x = data_test.values #returns a numpy array
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(data_x)
data_2 = pd.DataFrame(x_scaled,columns=list(data_test.columns))
data_2.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,0.007013,0.427806,0.058701,0.385726,0.100219,0.511964,0.420792,0.464204
1,0.076890,0.426392,0.099888,0.510273,0.059625,0.391442,0.419402,0.457428
2,0.006978,0.425638,0.060636,0.383772,0.099711,0.509370,0.418660,0.472388
3,0.076762,0.425683,0.099721,0.509424,0.056176,0.390791,0.418705,0.460715
4,0.006982,0.425911,0.058441,0.384018,0.094538,0.509697,0.418929,0.472691


In [942]:
y = data_2.loc[:,"Thickness"]
x = data_2.drop(columns="Thickness")
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [943]:
X_train

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T
248,0.056390,0.458170,0.105732,0.514560,0.105732,0.239658,0.422926
275,0.090179,0.423148,0.104053,0.534137,0.228916,0.409274,0.416211
68,0.062720,0.439038,0.103139,0.487820,0.055193,0.362381,0.418131
469,0.070842,0.453387,0.104067,0.290451,0.307312,0.510061,0.425051
63,0.006983,0.439926,0.057819,0.356130,0.101951,0.488806,0.418977
...,...,...,...,...,...,...,...
71,0.062706,0.438943,0.098240,0.487715,0.055460,0.362302,0.418041
106,0.098793,0.444568,0.103874,0.493964,0.056453,0.352831,0.423398
270,0.102047,0.414993,0.224504,0.523843,0.102047,0.401386,0.408190
435,0.073744,0.429053,0.300203,0.496093,0.229879,0.341902,0.402238


# 일반적으로 split을 진행하여

In [944]:
def rsq(y_pred,y_test):
    se = y_test - y_pred
    se2 = se ** 2
    s = sum(i for i in se2)

    st = y_test - np.mean(y_test)
    st2 = st ** 2
    t = sum(i for i in st2)

    r_sqr = 1 - (s/t)
    return r_sqr

In [945]:
tq =  tqdm(range(100))
n = 0
lr_result = []
for i in tq:
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    lr_result.append(list(lr.coef_)+[lr.intercept_])
    n += 1
lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
lr_statis = lr_df.describe()

  0%|          | 0/100 [00:00<?, ?it/s]

In [946]:
lr_df

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,intercept
0,-0.161502,-0.254137,-0.368421,-0.439484,-0.364151,-0.446525,-1.893878,1.836063
1,-0.179689,-0.199105,-0.373919,-0.428779,-0.359160,-0.444362,-1.895661,1.807992
2,-0.167993,-0.231709,-0.369785,-0.431893,-0.355922,-0.443847,-1.910638,1.828532
3,-0.165200,-0.246561,-0.357979,-0.436564,-0.367360,-0.445387,-1.898088,1.832186
4,-0.159046,-0.269292,-0.359559,-0.444071,-0.366393,-0.453757,-1.882219,1.842250
...,...,...,...,...,...,...,...,...
95,-0.169680,-0.250000,-0.371681,-0.449142,-0.366572,-0.459959,-1.865532,1.833139
96,-0.159111,-0.281217,-0.369809,-0.440950,-0.359833,-0.447181,-1.898051,1.850178
97,-0.159769,-0.202845,-0.366890,-0.432062,-0.371573,-0.435385,-1.919667,1.816832
98,-0.164875,-0.251913,-0.372067,-0.433152,-0.351467,-0.443400,-1.891471,1.828987


In [947]:
lr_statis

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,intercept
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-0.163418,-0.256982,-0.366228,-0.440219,-0.362312,-0.448820,-1.887291,1.835647
std,0.010317,0.026709,0.006607,0.008294,0.007306,0.007995,0.025493,0.011851
min,-0.185948,-0.316465,-0.384080,-0.466135,-0.380448,-0.478268,-1.937762,1.807992
25%,-0.171403,-0.273805,-0.370373,-0.443208,-0.366656,-0.451580,-1.902528,1.828799
50%,-0.163387,-0.254697,-0.365741,-0.439619,-0.363359,-0.448601,-1.888346,1.835725
75%,-0.155429,-0.242485,-0.361549,-0.434979,-0.357250,-0.443496,-1.872722,1.842252
max,-0.136687,-0.196083,-0.353192,-0.423055,-0.345364,-0.435149,-1.815168,1.870055


In [948]:
col_list = lr_statis.columns.to_list()
wei_data = []
## ANN 의 bias와 lr의 bias는 다르게 처리해야 한다.
for col_ in col_list:
    mean_ = lr_statis.loc["mean",col_]
    std_ = lr_statis.loc["std",col_]
    globals()["x_{}".format(col_)] = np.random.normal(mean_,std_,size=(1,3))
    wei_data.append(globals()["x_{}".format(col_)])

wei_test = np.vstack(wei_data).T

In [949]:
x_CP1

array([[-0.44214335, -0.44633415, -0.44865353]])

In [950]:
wei_1 = [we[:-1] for we in wei_test]
wei_1

[array([-0.16997836, -0.31514577, -0.34725076, -0.44214335, -0.36898119,
        -0.4533553 , -1.84805236]),
 array([-0.14899931, -0.27315478, -0.36683715, -0.44633415, -0.37748509,
        -0.45356563, -1.90768005]),
 array([-0.17318529, -0.27785858, -0.36468138, -0.44865353, -0.35424302,
        -0.45162601, -1.83415845])]

In [951]:
wei_test

array([[-0.16997836, -0.31514577, -0.34725076, -0.44214335, -0.36898119,
        -0.4533553 , -1.84805236,  1.83825579],
       [-0.14899931, -0.27315478, -0.36683715, -0.44633415, -0.37748509,
        -0.45356563, -1.90768005,  1.83501366],
       [-0.17318529, -0.27785858, -0.36468138, -0.44865353, -0.35424302,
        -0.45162601, -1.83415845,  1.84426517]])

In [952]:
def initial_weight(lr_statis,layers_other):
    col_list = lr_statis.columns.to_list()
    w_all = []
    w_candi = []
    n_layers = layers_other[0]
    
    for col_ in col_list:
        mean_ = lr_statis.loc["mean",col_]
        std_ = lr_statis.loc["std",col_]
        candi_ = np.random.normal(mean_,std_,size=(1,n_layers))
        w_candi.extend(candi_)
    
    w_stack = np.stack(w_candi)
    w_all.append(w_stack)
    wei_2 = [np.random.normal(loc=0.0, scale=0.1, size=(y, x)) for x, y in zip(layers_other[:-1], layers_other[1:])]
    
    w_all.extend(wei_2)
    
    return w_all

In [953]:
WIE = initial_weight(lr_statis,[3,1])

In [954]:
WIE[0]

array([[-0.15745785, -0.14789783, -0.18849325],
       [-0.23205153, -0.22843572, -0.27514285],
       [-0.36305589, -0.35489461, -0.36991479],
       [-0.44322861, -0.43199492, -0.44261955],
       [-0.37462968, -0.35145822, -0.36293538],
       [-0.45664404, -0.45086681, -0.44803358],
       [-1.91469964, -1.85161105, -1.91580625],
       [ 1.82555984,  1.85709659,  1.84321296]])

In [955]:
WIE

[array([[-0.15745785, -0.14789783, -0.18849325],
        [-0.23205153, -0.22843572, -0.27514285],
        [-0.36305589, -0.35489461, -0.36991479],
        [-0.44322861, -0.43199492, -0.44261955],
        [-0.37462968, -0.35145822, -0.36293538],
        [-0.45664404, -0.45086681, -0.44803358],
        [-1.91469964, -1.85161105, -1.91580625],
        [ 1.82555984,  1.85709659,  1.84321296]]),
 array([[ 0.04210378, -0.10932463,  0.07027207]])]

In [956]:
WIE[1]

array([[ 0.04210378, -0.10932463,  0.07027207]])

In [957]:
X_train.values

array([[0.03021902, 0.48350436, 0.11626769, ..., 0.12155602, 0.54394241,
        0.45328534],
       [0.0145646 , 0.46606707, 0.21257028, ..., 0.17812501, 0.33498571,
        0.43693788],
       [0.07296793, 0.42454068, 0.2970458 , ..., 0.22281752, 0.38473999,
        0.39800688],
       ...,
       [0.0733445 , 0.41339628, 0.10001523, ..., 0.06000914, 0.42673165,
        0.40006092],
       [0.00697321, 0.42536565, 0.06275887, ..., 0.10459811, 0.50904414,
        0.41839244],
       [0.04895436, 0.42660224, 0.09755904, ..., 0.05720666, 0.36366093,
        0.41960876]])

In [958]:
WIE[0]

array([[-0.15745785, -0.14789783, -0.18849325],
       [-0.23205153, -0.22843572, -0.27514285],
       [-0.36305589, -0.35489461, -0.36991479],
       [-0.44322861, -0.43199492, -0.44261955],
       [-0.37462968, -0.35145822, -0.36293538],
       [-0.45664404, -0.45086681, -0.44803358],
       [-1.91469964, -1.85161105, -1.91580625],
       [ 1.82555984,  1.85709659,  1.84321296]])

In [959]:
reluDerivative(np.dot(X,WIE[0]))

array([[1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [960]:
sigmoid(np.dot(X[0],WIE[0].T[0]))

0.9051486674789864

In [961]:
relu(np.dot(X,WIE[0]))

array([[2.25578846, 2.26685125, 2.29575265],
       [1.43386197, 1.48346736, 1.47612852],
       [0.        , 0.        , 0.        ],
       [2.07124429, 2.10061541, 2.02927098],
       [2.01988109, 2.03984685, 2.02412174],
       [2.001375  , 2.03168173, 1.92423523],
       [2.52192331, 2.53927039, 2.53004587],
       [1.33975504, 1.397737  , 1.47481439],
       [1.13418739, 1.22322019, 1.12892323],
       [2.20466525, 2.22243955, 2.20441768]])

In [962]:
np.dot(relu(np.dot(X,WIE[0])),WIE[1].T)

array([[ 0.00848185],
       [ 0.0019221 ],
       [ 0.        ],
       [ 0.00015929],
       [ 0.00427836],
       [-0.0026274 ],
       [ 0.00636928],
       [ 0.00723994],
       [-0.00664274],
       [ 0.00476636]])

In [963]:
WIE[1].shape

(1, 3)

In [964]:
relu(np.dot(X,WIE[0])).shape

(10, 3)

In [965]:
def relu(x) : 
    x[x<=0] = 0
    return x

def reluDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [966]:
# Define useful functions    

# Activation function


def relu(x) : 
    x[x<=0] = 0
    return x

def reluDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def sigmoid(t):
    return 1/(1+np.exp(-t))

# Derivative of sigmoid
def sigmoid_derivative(p):
    return p * (1 - p)

# Class definition
class NeuralNetwork:
    def __init__(self, x,y,WIE,learn):
        self.input = x
        self.weights1= WIE[0]
        self.weights2 = WIE[1]
        #self.weights3 = WIE[2]
        self.alpha = learn
        self.y = y
        self.output = np. zeros(y.shape)
        
    def feedforward(self):
        self.layer1 = relu(np.dot(self.input, self.weights1))
        self.layer2 = relu(np.dot(self.layer1, self.weights2.T))
        #self.layer3 = sigmoid(np.dot(self.layer2, self.weights3))
        
        return self.layer2
        
    def backprop(self):
        d2 = 2*(self.y -self.output)*reluDerivative(self.output)
        
        d_weights2 = np.dot(self.layer1.T, d2)
        d_weights1 = np.dot(self.input.T, np.dot(d2, self.weights2)*reluDerivative(self.layer1))

        
        
        self.weights1 += self.alpha*d_weights1
        self.weights2 += self.alpha*d_weights2.T

    def train(self, X, y):
        self.output = self.feedforward()
        self.backprop()


In [1]:

def sigmoid(t):
    return 1/(1+np.exp(-t))

# Derivative of sigmoid
def sigmoid_derivative(p):
    return p * (1 - p)

def relu(z):
    return np.maximum(0,z)

def d_relu(z):

    z[z>=0]=1
    z[z<0]= 0
    return z


def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - x * x


def none(z):
    return z

def d_none(z):
    return 1




class NeuralNetwork:
    def __init__(self, x,y,WIE,learn):
        self.input = x
        self.weights1= WIE[0]
        self.weights2 = WIE[1].T
        self.weights3 = WIE[2].T
        
        self.y = y
        self.alpha = learn
        self.output = np.zeros(y.shape)
        self.acti = sigmoid
        self.d_acti = sigmoid_derivative
        
        self.acti_out = sigmoid
        self.d_acti_out = sigmoid_derivative

    def feedforward(self):
        self.layer1 = self.acti(np.dot(self.input, self.weights1))
        self.layer2 = self.acti(np.dot(self.layer1, self.weights2))
        self.layer3 = self.acti_out(np.dot(self.layer2, self.weights3))
        
        return self.layer3
        
    def backprop(self):
        d_3 = 2*(self.y- self.output)*self.d_acti(self.output)
        d_2 = np.dot(d_3,self.weights3.T)*self.d_acti(self.layer2)
        d_1 = np.dot(d_2,self.weights2.T)*self.d_acti_out(self.layer1)
        
        d_weights3 = np.dot(d_3.T,self.layer2).T
        d_weights2 = np.dot(self.layer1.T,d_2)
        d_weights1 = np.dot(self.input.T,d_1)
        
        
        
        
        self.weights1 += self.alpha * d_weights1

        self.weights2 += self.alpha * d_weights2
        
        self.weights3 += self.alpha * d_weights3

    def train(self, X, y):
        self.output = self.feedforward()
        self.backprop()

In [968]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)[0:10]

y = np.array(y_train).reshape(-1,1).astype(np.float)[0:10]

In [970]:
WIE = initial_weight(lr_statis,[64,64,1])

In [971]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]]
Predicted Output: 
[[0.58672749]
 [0.58677233]
 [0.5866937 ]
 [0.58641341]
 [0.58630753]
 [0.58633281]
 [0.58630642]
 [0.58663676]
 [0.58636326]
 [0.58633683]]
Loss: 
0.021894771250503958


for iteration # 10

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]]
Predicted Output: 
[[0.46422155]
 [0.46433867]
 [0.46414785]
 [0.46348572]
 [0.46324963]
 [0.46330302]
 [0.46323984]
 [0.46401021]
 [0.46337881]
 [0.46331355]]
Loss: 
0.004325689996324746


for iteration # 20

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]]
Predicted Output: 
[[0.454894  ]
 [0.45501514]
 [0.45481805]
 [0.45413522]
 [0.45389198]
 

In [975]:
y.mean()

0.4534282895833804

In [976]:
NN.feedforward().mean()

0.45354464203713213

In [986]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

'0.00026%'

In [993]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)

y = np.array(y_train).reshape(-1,1).astype(np.float)

In [994]:
WIE = initial_weight(lr_statis,[100,64,1])

In [995]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]
 [0.48542366]
 [0.44463328]
 [0.47760598]
 [0.48079551]
 [0.47157607]
 [0.4796462 ]
 [0.49463176]
 [0.49896025]
 [0.41047881]
 [0.4786912 ]
 [0.49329107]
 [0.42767802]
 [0.48657392]
 [0.49655784]
 [0.51990898]
 [0.47979765]
 [0.46694301]
 [0.43315036]
 [0.58924783]
 [0.50992181]
 [0.47745443]
 [0.5167353 ]
 [0.44916269]
 [0.51295895]
 [0.51432387]
 [0.37776807]
 [0.53518824]
 [0.54038248]
 [0.37343536]
 [0.50399986]
 [0.5106595 ]
 [0.50058358]
 [0.58216137]
 [0.44768568]
 [0.49976507]
 [0.46040339]
 [0.49712432]
 [0.49148193]
 [0.35513625]
 [0.37758674]
 [0.41161615]
 [0.47680729]
 [0.52955966]
 [0.47953586]
 [0.40782973]
 [0.50178073]
 [0.39869738]
 [0.40869187]
 [0.36499407]
 [0.53973773]
 [0.47369875]
 [0.49477177]
 [0.50223851]
 [0.45377798]
 [0.42023372]
 [0.38123984]
 [0.50875746]
 [0.50145891]
 [0.43686345]

Loss: 
0.0027133864907311505


for iteration # 70

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]
 [0.48542366]
 [0.44463328]
 [0.47760598]
 [0.48079551]
 [0.47157607]
 [0.4796462 ]
 [0.49463176]
 [0.49896025]
 [0.41047881]
 [0.4786912 ]
 [0.49329107]
 [0.42767802]
 [0.48657392]
 [0.49655784]
 [0.51990898]
 [0.47979765]
 [0.46694301]
 [0.43315036]
 [0.58924783]
 [0.50992181]
 [0.47745443]
 [0.5167353 ]
 [0.44916269]
 [0.51295895]
 [0.51432387]
 [0.37776807]
 [0.53518824]
 [0.54038248]
 [0.37343536]
 [0.50399986]
 [0.5106595 ]
 [0.50058358]
 [0.58216137]
 [0.44768568]
 [0.49976507]
 [0.46040339]
 [0.49712432]
 [0.49148193]
 [0.35513625]
 [0.37758674]
 [0.41161615]
 [0.47680729]
 [0.52955966]
 [0.47953586]
 [0.40782973]
 [0.50178073]
 [0.39869738]
 [0.40869187]
 [0.36499407]
 [0.53973773]
 [0.47369875]
 [0.49477177]
 [0.50223851]
 [0.45377798]
 [0.42023372]
 [0.38123984]
 [0.50875

 [0.46322627]]
Loss: 
0.002698744667087575


for iteration # 140

Actual Output: 
[[0.37380931]
 [0.36768323]
 [0.37684619]
 [0.4805215 ]
 [0.52385196]
 [0.50198194]
 [0.50374749]
 [0.38456978]
 [0.52151946]
 [0.49975204]
 [0.48542366]
 [0.44463328]
 [0.47760598]
 [0.48079551]
 [0.47157607]
 [0.4796462 ]
 [0.49463176]
 [0.49896025]
 [0.41047881]
 [0.4786912 ]
 [0.49329107]
 [0.42767802]
 [0.48657392]
 [0.49655784]
 [0.51990898]
 [0.47979765]
 [0.46694301]
 [0.43315036]
 [0.58924783]
 [0.50992181]
 [0.47745443]
 [0.5167353 ]
 [0.44916269]
 [0.51295895]
 [0.51432387]
 [0.37776807]
 [0.53518824]
 [0.54038248]
 [0.37343536]
 [0.50399986]
 [0.5106595 ]
 [0.50058358]
 [0.58216137]
 [0.44768568]
 [0.49976507]
 [0.46040339]
 [0.49712432]
 [0.49148193]
 [0.35513625]
 [0.37758674]
 [0.41161615]
 [0.47680729]
 [0.52955966]
 [0.47953586]
 [0.40782973]
 [0.50178073]
 [0.39869738]
 [0.40869187]
 [0.36499407]
 [0.53973773]
 [0.47369875]
 [0.49477177]
 [0.50223851]
 [0.45377798]
 [0.42023372]
 [0.3812

In [996]:
y.mean()

0.4638243600340797

In [997]:
NN.feedforward().mean()

0.4628871159266004

In [998]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

'0.00202%'

# 다른 모델

# 'EM-370Z	' 로 test 하기

In [1040]:
data_test = data_drop[data_drop["MatlMaker"]=="NPG-151"][data_drop["MatlG"]==46.8]
#data_test = data_drop

drop_list = ['MatlMaker','MatlG',"Model","PP"]
data_test = data_test.drop(columns=drop_list)
data_test.head()

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
66,1,62.0,8.86,60,13.57,72,60,72.84
68,3,67.0,14.28,60,14.28,71,60,77.12
70,5,67.0,14.28,63,13.57,72,60,73.55
72,7,67.0,14.28,71,14.28,61,60,76.41
74,9,67.0,13.57,71,13.57,59,60,72.84


In [1041]:
data_x = data_test.values #returns a numpy array
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(data_x)
data_2 = pd.DataFrame(x_scaled,columns=list(data_test.columns))
data_2.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,0.006773,0.419943,0.060011,0.406396,0.091913,0.487676,0.406396,0.493365
1,0.019741,0.440882,0.093967,0.394819,0.093967,0.467203,0.394819,0.507475
2,0.032923,0.441165,0.094027,0.414827,0.089352,0.474088,0.395073,0.484294
3,0.046011,0.440386,0.093861,0.466678,0.093861,0.400949,0.394376,0.502238
4,0.060180,0.448003,0.090737,0.474750,0.090737,0.394510,0.401197,0.487053


In [1042]:
y = data_2.loc[:,"Thickness"]
x = data_2.drop(columns="Thickness")
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# 일반적으로 split을 진행하여

In [1043]:
def rsq(y_pred,y_test):
    se = y_test - y_pred
    se2 = se ** 2
    s = sum(i for i in se2)

    st = y_test - np.mean(y_test)
    st2 = st ** 2
    t = sum(i for i in st2)

    r_sqr = 1 - (s/t)
    return r_sqr

In [1044]:
tq =  tqdm(range(100))
n = 0
lr_result = []
for i in tq:
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    lr_result.append(list(lr.coef_)+[lr.intercept_])
    n += 1
lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
lr_statis = lr_df.describe()

  0%|          | 0/100 [00:00<?, ?it/s]

In [1045]:
def initial_weight(lr_statis,layers_other):
    col_list = lr_statis.columns.to_list()
    w_all = []
    w_candi = []
    n_layers = layers_other[0]
    
    for col_ in col_list:
        mean_ = lr_statis.loc["mean",col_]
        std_ = lr_statis.loc["std",col_]
        candi_ = np.random.normal(mean_,std_,size=(1,n_layers))
        w_candi.extend(candi_)
    
    w_stack = np.stack(w_candi)
    w_all.append(w_stack)
    wei_2 = [np.random.normal(loc=0.0, scale=0.1, size=(y, x)) for x, y in zip(layers_other[:-1], layers_other[1:])]
    
    w_all.extend(wei_2)
    
    return w_all

In [1046]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)[0:10]

y = np.array(y_train).reshape(-1,1).astype(np.float)[0:10]

In [1047]:
WIE = initial_weight(lr_statis,[64,64,1])

In [1048]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.50875746]
 [0.48442592]
 [0.49934778]
 [0.48676339]
 [0.50223755]
 [0.49241398]
 [0.49287524]
 [0.48725967]
 [0.4941182 ]
 [0.48885396]]
Predicted Output: 
[[0.52985442]
 [0.52961442]
 [0.52976033]
 [0.52963704]
 [0.52978774]
 [0.52968802]
 [0.52969254]
 [0.52963972]
 [0.52970651]
 [0.52965946]]
Loss: 
0.0013481720448738463


for iteration # 10

Actual Output: 
[[0.50875746]
 [0.48442592]
 [0.49934778]
 [0.48676339]
 [0.50223755]
 [0.49241398]
 [0.49287524]
 [0.48725967]
 [0.4941182 ]
 [0.48885396]]
Predicted Output: 
[[0.49613016]
 [0.49592738]
 [0.49605016]
 [0.49594652]
 [0.49607383]
 [0.49598902]
 [0.49599284]
 [0.49594844]
 [0.49600488]
 [0.49596565]]
Loss: 
5.770588596920024e-05


for iteration # 20

Actual Output: 
[[0.50875746]
 [0.48442592]
 [0.49934778]
 [0.48676339]
 [0.50223755]
 [0.49241398]
 [0.49287524]
 [0.48725967]
 [0.4941182 ]
 [0.48885396]]
Predicted Output: 
[[0.49397944]
 [0.49377908]
 [0.49390036]
 [0.493798  ]
 [0.49392378]

In [1049]:
y.mean()

0.49370531338963686

In [1050]:
NN.feedforward().mean()

0.49370779176542356

In [1051]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

'0.00001%'

In [1052]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)

y = np.array(y_train).reshape(-1,1).astype(np.float)

In [1053]:
WIE = initial_weight(lr_statis,[64,64,1])

In [1054]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.50875746]
 [0.48442592]
 [0.49934778]
 [0.48676339]
 [0.50223755]
 [0.49241398]
 [0.49287524]
 [0.48725967]
 [0.4941182 ]
 [0.48885396]
 [0.50701201]
 [0.48845818]
 [0.47603037]
 [0.51500479]
 [0.50178073]
 [0.50223911]
 [0.50733771]
 [0.50701201]
 [0.49477177]
 [0.49868194]
 [0.48748089]
 [0.49348333]
 [0.49336517]
 [0.48429401]
 [0.47567675]
 [0.48238597]
 [0.50187899]
 [0.48616573]
 [0.4878322 ]
 [0.48734088]
 [0.48269712]
 [0.48705299]
 [0.4930667 ]
 [0.505696  ]
 [0.50912741]
 [0.49641036]
 [0.48017451]
 [0.50747462]]
Predicted Output: 
[[0.48433246]
 [0.48468313]
 [0.48446068]
 [0.48465031]
 [0.48442996]
 [0.48456626]
 [0.48455966]
 [0.48464042]
 [0.4845427 ]
 [0.4846185 ]
 [0.48435853]
 [0.4846263 ]
 [0.48479757]
 [0.48423929]
 [0.48443521]
 [0.48442945]
 [0.48435481]
 [0.48435853]
 [0.48453452]
 [0.48447597]
 [0.48463777]
 [0.48454741]
 [0.48455305]
 [0.48468321]
 [0.48480199]
 [0.48471179]
 [0.48443378]
 [0.48465701]
 [0.48463193]
 [0.484

In [1055]:
y.mean()

0.4941311945055635

In [1056]:
NN.feedforward().mean()

0.4941359395555458

In [1057]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

'0.00001%'

# 다른 모델

# 'EM-370Z	' 로 test 하기

In [1076]:
data_test = data_drop[data_drop["MatlMaker"]=="EM-370Z"][data_drop["MatlG"]==46.8]
#data_test = data_drop

drop_list = ['MatlMaker','MatlG',"Model","PP"]
data_test = data_test.drop(columns=drop_list)
data_test.head()

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
249,1,63.0,8.14,50,14.91,70,60,67.10
254,6,63.0,15.66,72,15.66,43,60,53.44
257,9,63.0,16.40,44,13.42,72,60,55.66
262,14,63.0,14.17,70,8.30,50,60,67.84
263,1,63.0,8.30,50,15.66,70,60,64.86


In [1077]:
data_x = data_test.values #returns a numpy array
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(data_x)
data_2 = pd.DataFrame(x_scaled,columns=list(data_test.columns))
data_2.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,0.007114,0.448163,0.057906,0.355685,0.106065,0.497959,0.426822,0.477330
1,0.044741,0.469779,0.116774,0.536890,0.116774,0.320643,0.447409,0.398492
2,0.066499,0.465492,0.121176,0.325106,0.099157,0.531991,0.443326,0.411259
3,0.098904,0.445066,0.100105,0.494518,0.058636,0.353227,0.423873,0.479259
4,0.007163,0.451257,0.059451,0.358141,0.112170,0.501397,0.429769,0.464580


In [1078]:
y = data_2.loc[:,"Thickness"]
x = data_2.drop(columns="Thickness")
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# 일반적으로 split을 진행하여

In [1079]:
def rsq(y_pred,y_test):
    se = y_test - y_pred
    se2 = se ** 2
    s = sum(i for i in se2)

    st = y_test - np.mean(y_test)
    st2 = st ** 2
    t = sum(i for i in st2)

    r_sqr = 1 - (s/t)
    return r_sqr

In [1080]:
tq =  tqdm(range(100))
n = 0
lr_result = []
for i in tq:
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    lr_result.append(list(lr.coef_)+[lr.intercept_])
    n += 1
lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
lr_statis = lr_df.describe()

  0%|          | 0/100 [00:00<?, ?it/s]

In [1081]:
def initial_weight(lr_statis,layers_other):
    col_list = lr_statis.columns.to_list()
    w_all = []
    w_candi = []
    n_layers = layers_other[0]
    
    for col_ in col_list:
        mean_ = lr_statis.loc["mean",col_]
        std_ = lr_statis.loc["std",col_]
        candi_ = np.random.normal(mean_,std_,size=(1,n_layers))
        w_candi.extend(candi_)
    
    w_stack = np.stack(w_candi)
    w_all.append(w_stack)
    wei_2 = [np.random.normal(loc=0.0, scale=0.1, size=(y, x)) for x, y in zip(layers_other[:-1], layers_other[1:])]
    
    w_all.extend(wei_2)
    
    return w_all

In [1082]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)[0:10]

y = np.array(y_train).reshape(-1,1).astype(np.float)[0:10]

In [1083]:
WIE = initial_weight(lr_statis,[64,64,1])

In [1084]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.4804859 ]
 [0.40009757]
 [0.48724736]
 [0.4792588 ]
 [0.47369875]
 [0.40648537]
 [0.40511425]
 [0.40971643]
 [0.40068532]
 [0.39718192]]
Predicted Output: 
[[0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]
 [0.50588103]]
Loss: 
0.006608941923625759


for iteration # 10

Actual Output: 
[[0.4804859 ]
 [0.40009757]
 [0.48724736]
 [0.4792588 ]
 [0.47369875]
 [0.40648537]
 [0.40511425]
 [0.40971643]
 [0.40068532]
 [0.39718192]]
Predicted Output: 
[[0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]
 [0.43859859]]
Loss: 
0.0014628247452004087


for iteration # 20

Actual Output: 
[[0.4804859 ]
 [0.40009757]
 [0.48724736]
 [0.4792588 ]
 [0.47369875]
 [0.40648537]
 [0.40511425]
 [0.40971643]
 [0.40068532]
 [0.39718192]]
Predicted Output: 
[[0.4343081]
 [0.4343081]
 [0.4343081]
 [0.4343081]
 [0.4343081]
 [0.4

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [1085]:
y.mean()

0.4339971665857506

In [1086]:
NN.feedforward().mean()

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0.4339971665857506

In [1087]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


'0.00000%'

In [1088]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)

y = np.array(y_train).reshape(-1,1).astype(np.float)

In [1089]:
WIE = initial_weight(lr_statis,[64,64,1])

In [1090]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        #print(NN.weights1)
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Actual Output: 
[[0.4804859 ]
 [0.40009757]
 [0.48724736]
 [0.4792588 ]
 [0.47369875]
 [0.40648537]
 [0.40511425]
 [0.40971643]
 [0.40068532]
 [0.39718192]
 [0.40378146]
 [0.41051804]
 [0.40044604]
 [0.478105  ]
 [0.48071629]
 [0.47471907]
 [0.40178387]
 [0.47809392]
 [0.48059935]
 [0.48294181]
 [0.47634377]
 [0.47732953]
 [0.41125855]
 [0.40842327]
 [0.39804328]
 [0.49721624]
 [0.47819202]
 [0.48724736]
 [0.4079721 ]
 [0.46457998]
 [0.41161615]
 [0.40869187]
 [0.50111154]
 [0.48444307]
 [0.39849201]]
Predicted Output: 
[[0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.58106515]
 [0.581

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [1091]:
y.mean()

0.44407535054292396

In [1092]:
NN.feedforward().mean()

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0.44407535054292396

In [1093]:
"%.5f%%"%(abs((y.mean() - NN.feedforward().mean())/y.mean()))

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


'0.00000%'

# Loss function 문제